<a href="https://colab.research.google.com/github/abhilashhn1993/depression-assessment-on-twitter/blob/Dragon/AnchorCorexTopicModeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import string
import pandas as pd
import numpy as np
import datetime
import os
import pandas as pd
import re
import numpy as np
import pandas as pd
from pprint import pprint
import gensim
import spacy
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt
%matplotlib inline
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Tweet_timeline_analysis.xlsx to Tweet_timeline_analysis.xlsx


In [ ]:
import io
df_pre = pd.read_excel(io.BytesIO(uploaded['Tweet_timeline_analysis.xlsx']), sheet_name='pre_tweets')
df_post = pd.read_excel(io.BytesIO(uploaded['Tweet_timeline_analysis.xlsx']), sheet_name='post_tweets')

In [ ]:
df_pre = df_pre.iloc[:,[0,1,3]]
df_pre.head()

,Twitter_handle,Tweets,TimeSlot
0,__Siiwe,What is sprem production?,1
1,__Siiwe,This hopsital sucks,1
2,__Siiwe,I am seriously hurt,2
3,__Siiwe,I am in hospital and the bf has not visited me...,2
4,__Siiwe,LOL this escalated quickly,4


In [ ]:
df_post = df_post.iloc[:,[0,1,3]]
df_post.head()

,Twitter_handle,Tweets,TimeSlot
0,__Siiwe,@Uber might have its problems but taxify is re...,89
1,__Siiwe,Just one season,88
2,__Siiwe,See bae in a few hours,88
3,__Siiwe,I need a bursary to go study Risk management a...,88
4,__Siiwe,You guys clearly don't know what depression is...,88


In [ ]:
df_pre['Tweets'] = df_pre['Tweets'].map(lambda x: str(x))
df_post['Tweets'] = df_post['Tweets'].map(lambda x: str(x))

In [ ]:
import nltk;
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
stop_words.remove("don't")
stop_words.remove("been")
stop_words.remove("being")
stop_words.remove("haven't")
stop_words.remove("couldn't")
stop_words.remove("didn't")
stop_words.extend(['http', 'https', 'www'])

In [ ]:
import string
#METHODS FOR TEXT CLEANING#

#text punctuation removal
def remove_punctuation(text):
  for p in string.punctuation:
    text = text.replace(p,'')
  return text

#method for pre-processing
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

# Method for stopwords removal
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

#Method for text Lemmatization with POS tagging
nlp = spacy.load('en', disable=['parser', 'ner'])
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

#Method to remove texts with consecutive characters
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
    return input_txt

#Method to convert the cleaned tweets back into string
def convert_to_string(df):
  for row in range(len(df)):
    df.iloc[row].Tweets = ' '.join([str(element) for element in df.iloc[row].Tweets])
  return df

In [ ]:
def cleanTweets(df):
	# Convert to list
	df['Tweets'] = df.Tweets.values.tolist()

	# Remove Emails
	df['Tweets'] = [re.sub('\S*@\S*\s?', '', sent) for sent in df['Tweets']]
	# Remove new line characters
	df['Tweets'] = [re.sub('\s+', ' ', sent) for sent in df['Tweets']]
	# Remove Punctuations
	df['Tweets'] = df.Tweets.apply(remove_punctuation)
	# Remove distracting single quotes
	df['Tweets'] = [re.sub("\'", "", sent) for sent in df['Tweets']]
	# Remove consecutive characters
	df['Tweets'] = np.vectorize(remove_pattern)(df['Tweets'], "@[\w]*")

	df['Tweets'] = list(sent_to_words(df['Tweets']))
	df['Tweets'] = remove_stopwords(df['Tweets'])

	# Initialize spacy 'en' model
	df['Tweets'] = lemmatization(df['Tweets'], allowed_postags=['NOUN','ADJ','VERB','ADV'])

	# remove the stopwords again after lemmatizing the text
	df['Tweets'] = remove_stopwords(df['Tweets'])

	df = convert_to_string(df)
	df = df.drop([0], axis=0)
	return df

**Cleaning the Tweets**

In [ ]:
df_pre = cleanTweets(df_pre)
df_post  = cleanTweets(df_post)

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [ ]:
#Just in case pandas encounters the SettingWithCopyWarning 
df_pre['Tweets'] = df_pre['Tweets'].str.join(" ")
df_post['Tweets'] = df_post['Tweets'].str.join(" ")
df_pre.head()

,Twitter_handle,Tweets,TimeSlot
1,__Siiwe,suck,1
2,__Siiwe,seriously hurt,2
3,__Siiwe,visit must say extremely disappointed,2
4,__Siiwe,escalate quickly,4
5,__Siiwe,shop need work shoe,4


In [ ]:
df_post.head()

,Twitter_handle,Tweets,TimeSlot
1,__Siiwe,season,88
2,__Siiwe,see bae hour,88
3,__Siiwe,need bursary go study risk management complian...,88
4,__Siiwe,guy clearly know depression blame death ignora...,88
5,__Siiwe,sleep classified hobbie,87


**COREX Topic Modeling**

In [ ]:
!pip install corextopic
from corextopic import corextopic as ct
from corextopic import vis_topic as vt
import scipy.sparse as ss
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vectorizer = CountVectorizer(stop_words='english', max_features=None, binary=True, ngram_range=(1,2))
doc_word = vectorizer.fit_transform(df_pre['Tweets'])
doc_word = ss.csr_matrix(doc_word)
doc_word.shape

(125157, 256678)

125157 docs(rows of data in the original dataframe) and 256678 words/features in the **doc-word matrix**

In [ ]:
# Get words that label the columns (needed to extract readable topics and make anchoring easier)
words = list(np.asarray(vectorizer.get_feature_names()))

In [ ]:
# Train the CorEx topic model with 50 topics
topic_model = ct.Corex(n_hidden=7, words=words, max_iter=200, verbose=False, seed=1)
topic_model.fit(doc_word, words=words);

In [ ]:
# Print all topics from the CorEx topic model
topics = topic_model.get_topics()
for n,topic in enumerate(topics):
    topic_words,_ = zip(*topic)
    print('{}: '.format(n) + ','.join(topic_words))

0: woman,black,man,little,parent,wear,body,run,entire,season
1: watch,movie,ass,twitter mention,mention like,watch movie,voice,ice cream,mention,ice
2: people,make,know,feel,think,say,want,really,thing,time
3: meet,adore,absolutely adore,fave,meet fave,hope meet,wait meet,fucking,fave soon,support fave
4: happy,birthday,happy birthday,sense,make sense,make happy,look forward,spend,forward,great
5: work,look,new,good,ahh,art,sleep,tomorrow,night,lovely
6: love,hope,deserve,amazing,support,girl,thank,stay,smile,soon


In [ ]:
topic_model.get_topics(n_words=10, topic=1)

[('watch', 0.002768583943508822),
 ('movie', 0.0018301352939584419),
 ('ass', 0.001290018848759047),
 ('twitter mention', 0.0012309785225930612),
 ('mention like', 0.0011760519128503227),
 ('watch movie', 0.0011506364888965504),
 ('voice', 0.001002643091226039),
 ('ice cream', 0.0009838373064098031),
 ('mention', 0.0009131660214132437),
 ('ice', 0.0009120961001345846)]

In [ ]:
print(np.sum(topic_model.tcs))

5.054591336836067


**Anchored Corex: A semi-supervised approach**

In [ ]:
anchor_words = [
#LIFESTYLE 
['games','music','sports','video', 'care', 'pet', 'dog', 'party', 'event','fb','facebook','instagram','twitter','sing',
  'play station','ps','book','tysm','hope','art','art style'],
#MENTAL SYMPTOMS
['lonely', 'guilt','worthless', 'helpless','hopeless','overreacting','empty','suicidal','feel bad','low mood','anxiety','worry','harm','disturbed','agitated','irritable','suicidal thoughts',
  'suicide','end life','no mood','cry','depressed','irritation','lost','sad','cut','hurt','hate','fear','anger','angry','frustrated','panic','panic attack','depression','anxiety','mental',
  'cannot','cant','not happy','mood','pissed','afraid','upset','stress','low','feeling empty','feeling garbage','insecure','out of bed'],
#CAUSES
['abuse', 'assault', 'sexual assault','harassment','torture', 'rape', 'trauma', 'accident', 'death', 'substance abuse', 'drugs', 'alcohol','break up', 'ditched', 'divorce','gambling','flashbacks',
  'war','attack', 'suffer', 'genetic','family problem','fire','lost job','conflict', 'relocate', 'moved'],
#SWEAR WORDS & CURSING
['shit','fuck','fuck you','fucking','dafuq','fck','holy shit','oh god','damn','trash', 'ass', 'goddamn', 'hell', 'asshole', 'bitch', 'whore', 'slut', 'Jesus Christ', 'shitass', 'cunt', 'balls',
  'ballsack','crap', 'bullshit', 'bugger','bastard','sucked up', 'sucked','Jesus fuck'],
#PHYSICAL SYMPTOMS
['pain','headache','migraine','body pain','back pain','nightmare','insomnia','no sleep','constipation','ache','fatigue','low energy','laze','lathargic','soreness','aching','vision','weight loss',
  'weight gain','digestive problem','cramps','over eating','tired','eyes','getting fat', 'apetite', 'vomit', 'nausea','flu','cough','cold','allergy'],
#TREATMENT
['therapy','counselling','doctor','therapist','shrink','antidepressants','pills','meds','medicines','supplements','essential oils','psychotherapy','behavioral therapy','cognitive', 
  'counselor','psychologist','psychiatrist','hospital','clinic', 'celexa','prozac','zoloft','viibryd','lexapro','therapist','drug','rx'],
#COPING
['exercise','gym','work out','fitness','yoga','healthy','diet','meditation','support group', 'motivation','hobby','family','parents','goals','wife','husband','sunshine', 'support',
  'pamper','get help','charity','cuddle','cure','heal']]

In [ ]:
anchored_topic_model = ct.Corex(n_hidden=7, seed=2)
anchored_topic_model.fit(doc_word, words=words, anchors=anchor_words, anchor_strength=6);

In [ ]:
topic_words,_ = zip(*anchored_topic_model.get_topics(topic=1))
print('0: ' + ','.join(topic_words))

0: hate,sad,hurt,cut,anxiety,mental,mood,stress,worry,depression


In [ ]:
print(np.sum(anchored_topic_model.tcs))

7.790454631744797


In [ ]:
for n in range(len(anchor_words)):
    topic_words,_ = zip(*anchored_topic_model.get_topics(topic=n))
    print('{}: '.format(n) + ','.join(topic_words))

0: hope,video,art,care,twitter,music,tysm,book,dog,art style
1: hate,sad,hurt,cut,anxiety,mental,mood,stress,worry,depression
2: attack,death,abuse,war,suffer,accident,rape,alcohol,torture,assault
3: fuck,shit,fucking,bitch,ass,damn,hell,trash,asshole,crap
4: tired,pain,cold,cough,vomit,nightmare,vision,headache,allergy,ache
5: people,make,know,think,time,really,thing,say,want,feel
6: support,family,wife,husband,healthy,gym,motivation,heal,diet,cure


In [ ]:
vt.vis_rep(topic_model, column_label=words, prefix='twenty')